In [1]:
import cv2
import os
import tensorflow as tf 
import tensorflow.keras.models
from matplotlib import pyplot as plt
import numpy as np
from keras import models
import sys
from PIL import Image
import winsound
import time
import math

Load the Model 

In [2]:
model = models.load_model('models/transferCNN.h5',  compile=False)

In [3]:
faceCascade = cv2.CascadeClassifier('data/haarcascade/haarcascade_frontalface_default.xml')
eyeCascade = cv2.CascadeClassifier('data/haarcascade/haarcascade_eye.xml')

In [4]:
#vid = cv2.VideoCapture(0)

In [5]:
closed_retrain_output = 'closed'
open_retrain_output = 'open'
if not os.path.exists(closed_retrain_output):
    os.makedirs(closed_retrain_output)
if not os.path.exists(open_retrain_output):
    os.makedirs(open_retrain_output)

In [7]:
import cv2

path = "data/haarcascade/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier('data/haarcascade/haarcascade_frontalface_default.xml')

vid = cv2.VideoCapture(0)
if not vid.isOpened():
    vid = cv2.VideoCapture(0)
if not vid.isOpened():
    raise IOError("Can't detect the camera")

frame_number = 0
counter = 3
start_time = time.time()
while vid.isOpened():
    #facemarkers = cv2.CascadeClassifier('data/haarcascade/haarcascade_frontalface_default.xml')
    ret, frame = vid.read()
    #filename = os.path.join(closed_retrain_output, f'frame_{frame_number}.jpg')  # Construct full path
    #cv2.imwrite(filename, frame)
    eyeCascade = cv2.CascadeClassifier('data/haarcascade/haarcascade_eye.xml')
    #frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    text = cv2.FONT_HERSHEY_SIMPLEX
    
    eyes = eyeCascade.detectMultiScale(gray, 1.1, 4)
    for x, y, w, h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        marked_eye = eyeCascade.detectMultiScale(roi_gray)
        if len(marked_eye) == 0:
            print("Eye not found!!")
        else:
            for(ex, ey, ew, eh) in marked_eye:
                eyes_roi = roi_color[ey: ey+eh, ex:ex+ew]
                
    
    resize_img =cv2.resize(eyes_roi, (224, 224))
    final_img = np.expand_dims(resize_img, axis=0)    
    final_pic = final_img/255.0
    #final_img = np.expand_dims(final_pic, axis=0)
    prediction = model.predict(final_pic)
    if(prediction > 0.5):
        #filename = f'frame_{frame_number}.jpg'
        #success = cv2.imwrite(filename, frame)
        filename = os.path.join("open", f'frame_{frame_number}.jpg')  # Construct full path
        #filename = "OpenCV_frame{}.png".format(frame_number)
        capture = cv2.imwrite(filename, frame)
        frame_number += 1
        if capture:
            print(f"Frame saved successfully as {filename}.")
        else:
            print(f"Error: Failed to save frame as {filename}.")
        
        outPut = "Open Eye "
        #cv2.putText(frame, outPut, (20,60), text, 3, (0, 255, 0), 2, cv2.LINE_4)
        x1, y1, w1, h1 = 0, 0, 180, 75
        #Drawing a black rectange
        cv2.rectangle(frame, (x1, y1), (x1+w1, y1+h1), (0, 0, 0), -1)
        #Adding text
        cv2.putText(frame, 'Active', (x1+ int(w1/10), y1+ int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        start_time = time.time()
    else:
        #counter +=1
        filename = os.path.join("closed", f'frame_{frame_number}.jpg')  # Construct full path
        capture = cv2.imwrite(filename, frame)
        frame_number += 1
        if capture:
            print(f"Frame saved successfully as {filename}.")
        else:
            print(f"Error: Failed to save frame as {filename}.")
        
        outPut = "Closed Eye"
        #cv2.putText(frame, outPut, (150, 150), text, 3, (0, 0, 255), 2, cv2.LINE_4)
        x1, y1, w1, h1 = 0, 0, 180, 75
        cv2.rectangle(frame, (x1, y1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Active', (x1+ int(w1/10), y1+ int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        if (int(time.time()-start_time) >= counter):
            x1, y1, w1, h1 = 0, 0, 180, 75
            cv2.rectangle(frame, (x1, y1), (x1+w1, y1+h1), (0, 0, 0), -1)
            cv2.putText(frame, 'Sleep Alert!!', (x1+ int(w1/10), y1+ int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            #winsound.axis(frequency=37, duration=5000)
            #counter = 0
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #print(faceCascade.empty())
    #faces = faceCascade.detectMultiScale(frame, 1.1, 4)
    
    #for(x, y, w, h) in faces:
    #    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    #tut = 'TutorialsPoint'
    image = cv2.putText(frame, outPut, (10, 65), text, 1, (255, 0, 0), 2, cv2.LINE_4)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

Eye not found!!
1/1 [==============================] - 1s 719ms/step
Frame saved successfully as open\frame_0.jpg.
Eye not found!!
Eye not found!!
1/1 [==============================] - 0s 56ms/step
Frame saved successfully as open\frame_1.jpg.
Eye not found!!
1/1 [==============================] - 0s 64ms/step
Frame saved successfully as closed\frame_2.jpg.
1/1 [==============================] - 0s 52ms/step
Frame saved successfully as closed\frame_3.jpg.
1/1 [==============================] - 0s 40ms/step
Frame saved successfully as closed\frame_4.jpg.
1/1 [==============================] - 0s 51ms/step
Frame saved successfully as closed\frame_5.jpg.
Eye not found!!
1/1 [==============================] - 0s 41ms/step
Frame saved successfully as closed\frame_6.jpg.
1/1 [==============================] - 0s 50ms/step
Frame saved successfully as closed\frame_7.jpg.
Eye not found!!
Eye not found!!
1/1 [==============================] - 0s 47ms/step
Frame saved successfully as closed\fram